In [4]:
import numpy as np, pandas as pd
from matplotlib import pyplot as plt
from scipy.optimize import minimize
import scipy.stats as stats
from matplotlib.ticker import FuncFormatter
import scipy.optimize as op
import os
from nltk import flatten

We use the same function in the mutually exciting process model of arrival and departure at station i, to filter the event times in process j that occurred before event time T in station i.

In [14]:
def prev_event(times, T):
    """
    Return the event times in process j that happens before event time T in process i.
    : param times: an n-dimensional array, the full event times in process j
    : param T: a number, one event time in process i
    
    : return k: a number, the index of the last event time in process j happened before T in process i
    """
    
    return np.searchsorted(times, T, side='right')

And again, similar to previous model, we use a dictionary to store all the filtered event times in each process j = 1, ..., m. We take T as the last event time observed in process i for our chosen piece of data. We label i as j=1.

In [30]:
def process_times(location_codes, times, i):
    """
    Return the departure and arrival times observed at station j = 1, ..., m, before the last event time
    observed in departure process at station i.
    
    : param location_codes: a list of length M, the location codes of all M stations, with i as the first.
    : param times: a dictionary, storing the departure and arrival times from station j = 1, ..., M as values, \
    and the location codes of corresponding stations as keys.
    : param i: the index of the station being studied
    
    : return times: a dictionary, storing all relevant event times
    """
    
    ii = get_station_index(location_codes, i)
    arr_times = times['arrival']
    dep_times = times['departure']
    T = dep_times[i][-1]
    ind_list = []
    ind1_list = []
    new_times = {}
    new_deptimes={}
    new_arrtimes={}
    
    for j in range(len(location_codes)):
        ind = prev_event(dep_times[location_codes[j]], T)
        if j != ii and ind != 0:
            new_deptimes[location_codes[j]] = dep_times[location_codes[j]][:ind]
        
        if ind == 0:
            ind_list.append(location_codes[j])
        
        if j == ii:
            new_deptimes[location_codes[j]] = dep_times[location_codes[j]]
        
        ind1 = prev_event(arr_times[location_codes[j]], T)
        if ind1 != 0:
            new_arrtimes[location_codes[j]] = arr_times[location_codes[j]][:ind1]
        
        if ind1 == 0:
            ind1_list.append(location_codes[j])
     
    new_times['arrival'] = new_arrtimes
    new_times['departure'] = new_deptimes
    
    return new_times, ind_list, ind1_list

For log likelihood, we still follow the previous model, to get function for $A_{ii}(1)$, ..., $A_{ii}(h)$

In [16]:
def A_ii(theta, times_i):
    """
    Finds the array of function A_i, from 1st event to hth event in process i
    
    : param theta: a real number
    : param times_i, a 1-D array, the event times observed in process i
    
    : return A: a 1D array
    """
   
    A = np.zeros(len(times_i))
    
    for h in range(1, len(times_i)):
        A[h] = np.exp(-theta*(times_i[h] - times_i[h-1]))*(1+A[h-1])
        
    return A

In [17]:
A_ii(2, np.array([1, 2, 3]))

array([0.        , 0.13533528, 0.15365092])

And for $A_{ij}(1)$, ..., $A_{ij}(h)$

In [18]:
def A_ij(theta, times_i, times_j):
    """
    Finds the array of function A_ij, for all event times in process j = 1, ..., M
    
    : param theta: a real number
    : param times_i, a 1-D array, the event times observed in process i
    : param times_j, a 1-D array, the event times observed in process j
    
    : return B: a 1-D array
    """
    
    B = np.zeros(len(times_i))
    ind = prev_event(times_j, times_i[0])
    
    B[0] = np.sum(np.exp(- theta * (times_i[0] - times_j[:ind])))
    
    for h in range(1, len(times_i)):
        B[h] = np.exp(-theta * (times_i[h] - times_i[h-1])) * B[h-1] 
        new_ind = prev_event(times_j, times_i[h])
        if ind != new_ind:
            B[h] += np.sum(np.exp(- theta * (times_i[h] - times_j[ind:new_ind])))
            ind = new_ind#prev_event(times_j, times_i[h])
        
    return B

We also write a function to find the excitation terms of each station

In [179]:
def excitation_j(theta, t, times_j):
    """
    Finds the array of function excitation_j, for all event times in process j = 1, ..., M
    
    : param theta: a real number, thetaj
    : param times_i, a 1-D array, the event times observed in process i
    : param times_j, a 1-D array, the event times observed in process j
    
    : return Ej: a 1-D array
    """
    #Ej = 0
    ind = prev_event(times_j, t)
    Ej = np.zeros(ind)
    if ind != 0:
            Ej = np.exp(-theta*(t - times_j[0:ind])) - 1
    #for i in range(ind):
        #Ej += np.exp(-theta*(t - times_j[i])) - 1
    
    return Ej

And log likelihood is hence

In [211]:
def seven_comp(t, location_codes, event_times, beta_dep, theta_dep, beta_arr, theta_arr, k_dep, k_arr, lambda_b, dist, i): 
    
    arr_times = event_times["arrival"]
    dep_times = event_times["departure"]
    times_i = arr_times[i]
    kappa_dep = kappa_fun(dist, k_dep)
    kappa_arr = kappa_fun(dist, k_arr)
    ratio_dep = kappa_dep*beta_dep/theta_dep
    ratio_arr = kappa_arr*beta_arr/theta_arr
    
    ex_terms = 0
    
    for j in range(len(location_codes)):
        times_dep = dep_times[location_codes[j]]
        times_arr = arr_times[location_codes[j]]
        ind_dep = prev_event(times_dep, t)
        ind_arr = prev_event(times_arr, t)
        
        if ind_dep != 0:
            ex_terms += ratio_dep[j] * np.sum(excitation_j(theta_dep, t, times_dep[0:ind_dep]))
        if ind_arr != 0:
            ex_terms += ratio_arr[j] * np.sum(excitation_j(theta_arr, t, times_arr[0:ind_arr]))
    
    res = lambda_b*t - ex_terms 
    
    return res

In [200]:
def seven_log_likelihood(location_codes, event_times, beta_dep, theta_dep, beta_arr, theta_arr, 
                        k_dep, k_arr, lambda_b, dist, i): 
    """
    Finds the log-likelihood of the mutually exciting process between stations
    
    : param location_codes: a list of length M, the location codes of all M stations with i as the first
    : param event_times: a dictionary, the event times in all processes \
      that occurred before the last event in process i 
    : param beta: a real number
    : param theta: a real number
    : param k: a real number
    : param dist: a 1-D array of length M, which stores the distances of each station
    : param lambda_b: a real number, the baseline intensity of station i
    : param i: the index of the station being studied
    
    : return A: a real number, the baseline intensity of station i
    """
    ind = i#ind = get_station_index(location_codes, i)
    arr_times = event_times["arrival"]
    dep_times = event_times["departure"]
    times_i = dep_times[i]
    
    kappa_dep = kappa_fun(dist, k_dep)
    kappa_arr = kappa_fun(dist, k_arr)
    ratio_dep = kappa_dep*beta_dep/theta_dep
    ratio_arr = kappa_arr*beta_arr/theta_arr
    T = times_i[-1]
    
    #Depature at station i
    A = A_ii(theta_dep, times_i)
    A = kappa_dep[ind]*beta_dep*A
    ex_terms = ratio_dep[ind] * np.sum(excitation_j(theta_dep, T, times_i))
    #Arrival at station i
    A += kappa_arr[ind]*beta_arr*A_ij(theta_arr, times_i, arr_times[location_codes[ind]])
    ex_terms += ratio_arr[ind] * np.sum(excitation_j(theta_arr, T, arr_times[location_codes[ind]]))
    
    for j in range(len(location_codes)):
        if j != ind:
            #Depature at station j
            A += kappa_dep[j] * beta_dep * A_ij(theta_dep, times_i, dep_times[location_codes[j]])
            ex_terms += ratio_dep[j] * np.sum(excitation_j(theta_dep, T, dep_times[location_codes[j]]))
            #Arrival at station j
            A += kappa_arr[j] * beta_arr * A_ij(theta_arr, times_i, arr_times[location_codes[j]])
            ex_terms += ratio_arr[j] * np.sum(excitation_j(theta_arr, T, arr_times[location_codes[j]]))
    
    res = np.sum(np.log(lambda_b +A)) + ex_terms - lambda_b*T
    
    return res

In [201]:
def seven_log_likelihood2(location_codes, event_times, beta_dep, theta_dep, beta_arr, theta_arr, 
                        k_dep, k_arr, lambda_b, dist, i): 
    """
    Finds the log-likelihood of the mutually exciting process between stations
    
    : param location_codes: a list of length M, the location codes of all M stations with i as the first
    : param event_times: a dictionary, the event times in all processes \
      that occurred before the last event in process i 
    : param beta: a real number
    : param theta: a real number
    : param k: a real number
    : param dist: a 1-D array of length M, which stores the distances of each station
    : param lambda_b: a real number, the baseline intensity of station i
    : param i: the index of the station being studied
    
    : return A: a real number, the baseline intensity of station i
    """
    ind = i#ind = get_station_index(location_codes, i)
    arr_times = event_times["arrival"]
    dep_times = event_times["departure"]
    times_i = dep_times[i]
    
    kappa_dep = kappa_fun(dist, k_dep)
    kappa_arr = kappa_fun(dist, k_arr)
    ratio_dep = kappa_dep*beta_dep/theta_dep
    ratio_arr = kappa_arr*beta_arr/theta_arr
    T = times_i[-1]
    ex_list = []
    
    #Depature at station i
    A = A_ii(theta_dep, times_i)
    A = kappa_dep[ind]*beta_dep*A
    ex_terms = ratio_dep[ind] * np.sum(excitation_j(theta_dep, T, times_i))
    ex_list.append(ratio_dep[ind] * np.sum(excitation_j(theta_dep, T, times_i)))
    #Arrival at station i
    A += kappa_arr[ind]*beta_arr*A_ij(theta_arr, times_i, arr_times[location_codes[ind]])
    ex_terms += ratio_arr[ind] * np.sum(excitation_j(theta_arr, T, arr_times[location_codes[ind]]))
    ex_list.append(ratio_arr[ind]*np.sum(excitation_j(theta_arr, T, arr_times[location_codes[ind]])))
    
    for j in range(len(location_codes)):
        if j != ind:
            #Depature at station j
            A += kappa_dep[j] * beta_dep * A_ij(theta_dep, times_i, dep_times[location_codes[j]])
            ex_terms += ratio_dep[j] * np.sum(excitation_j(theta_dep, T, dep_times[location_codes[j]]))
            ex_list.append(ratio_dep[j] * np.sum(excitation_j(theta_dep, T, dep_times[location_codes[j]])))
            #Arrival at station j
            A += kappa_arr[j] * beta_arr * A_ij(theta_arr, times_i, arr_times[location_codes[j]])
            ex_terms += ratio_arr[j] * np.sum(excitation_j(theta_arr, T, arr_times[location_codes[j]]))
            ex_list.append(ratio_arr[j] * np.sum(excitation_j(theta_arr, T, arr_times[location_codes[j]])))
    
    res = np.sum(np.log(lambda_b +A)) + ex_terms - lambda_b*T
    
    return np.sum(np.log(lambda_b +A)), np.array(ex_list), lambda_b*T, - ex_terms + lambda_b*T

We define the threshold function to select stations from a neighbourhood around the given station, at a threshold distance.

In [183]:
def thres_fun(dist, thres, loc_codes):
    new_dist = dist.copy()
    new_loc_codes = np.asarray(loc_codes).copy()
    
    index = [i for i in range(len(dist)) if dist[i] > thres]
    new_dist[index] = -1
    new_loc_codes[index] = -1
    new_dist = [i for i in new_dist if i != -1]
    new_loc_codes = [i for i in new_loc_codes if i != -1]
    
    return np.asarray(new_dist), new_loc_codes

And the kappa function.

In [184]:
def kappa_fun(dist, k):
    
    return np.exp(-k * dist)

In [185]:
dist = np.array([1, 2, 3, 4, 5])
k = 1
thres = 3
loc_codes = [4, 5, 6, 7, 8]
thres_fun(dist, thres, loc_codes)

(array([1, 2, 3]), [4, 5, 6])

In [210]:
beta_dep = 2.0
beta_arr = 1.5
theta_dep = 3.0
theta_arr = 3.5
dist = np.array([0, 0.5])
k_dep = 0.5
k_arr = 0.7
kappa_dep = kappa_fun(dist, k_dep)
kappa_arr = kappa_fun(dist, k_arr)
lambda_b = 1.0
location_codes = [0, 1]

t0_dep = np.sort(np.random.uniform(0,75,size=3))
t0_arr = np.sort(np.random.uniform(0,75,size=3))#np.array([1.0, 2.0, 3.0])
t1_dep = np.sort(np.random.uniform(0,75,size=3))#np.array([43.0, 57.0, 62.0])
t1_arr = np.sort(np.random.uniform(0,75,size=3))
t_dep = {0: t0_dep, 1: t1_dep}
t_arr = {0: t0_arr, 1: t1_arr}
times = {'departure': t_dep, 'arrival': t_arr}

tdict = process_times(location_codes, times, 0)[0]
t0_dep0 = tdict['departure'][0]
t0_arr0 = tdict['arrival'][0]
t1_dep0 = tdict['departure'][1]
t1_arr0 = tdict['arrival'][1]

sum_1 = []
for i in range(3):
    log_term = lambda_b 
    if i == 1:
        log_term += beta_dep*np.exp(-theta_dep*(t0_dep0[1] -t0_dep0[0]))
    
    if i == 2:
        log_term += beta_dep*np.exp(-theta_dep*(t0_dep0[2] -t0_dep0[0])) + beta_dep*np.exp(-theta_dep*(t0_dep0[2] -t0_dep0[1]))
    
    ind = np.searchsorted(t0_arr0, t0_dep0[i], side='right')
    for j in range(ind):
        log_term += kappa_arr[0]*beta_arr*np.exp(-theta_arr*(t0_dep0[i] - t0_arr0[j]))
        
    ind = np.searchsorted(t1_dep0, t0_dep0[i], side='right')
    for j in range(ind):
        log_term += kappa_dep[1]*beta_dep*np.exp(-theta_dep*(t0_dep0[i] - t1_dep0[j]))
    
    ind = np.searchsorted(t1_arr0, t0_dep0[i], side='right')
    for j in range(ind):
        log_term += kappa_arr[1]*beta_arr*np.exp(-theta_arr*(t0_dep0[i] - t1_arr0[j]))
    
    sum_1.append(np.log(log_term))

sum_2, sum_3, sum_4, sum_5 = 0.0, 0.0, 0.0, 0.0
for i in range(3):
    sum_2 += np.exp(-theta_dep*(t0_dep0[-1]-t0_dep0[i]))-1

index = np.searchsorted(t0_arr0, t0_dep0[-1], side="Right")
sum3 = []
for m in range(index):
    sum_3 += np.exp(-theta_arr*(t0_dep0[-1]-t0_arr0[m]))-1
    sum3.append(sum_3)
    a = excitation_j(theta_arr, t0_dep0[-1], t0_arr0)

index = np.searchsorted(t1_dep0, t0_dep0[-1], side="Right")
for m in range(index):
    sum_4 += np.exp(-theta_dep*(t0_dep0[-1]-t1_dep0[m]))-1

index = np.searchsorted(t1_arr0, t0_dep0[-1], side="Right")
for m in range(index):
    sum_5 += np.exp(-theta_arr*(t0_dep0[-1]-t1_arr0[m]))-1
    
sum_Lambda = np.array([kappa_dep[0]*beta_dep/theta_dep * sum_2, kappa_arr[0]*beta_arr/theta_arr * sum_3,
                       kappa_dep[1]*beta_dep/theta_dep * sum_4, kappa_arr[1]*beta_arr/theta_arr * sum_5])

l = np.sum(sum_1) - lambda_b * t0_dep0[-1] + np.sum(sum_Lambda)
u = seven_log_likelihood2(location_codes, times, beta_dep, theta_dep, beta_arr, theta_arr, k_dep, k_arr, lambda_b, dist, 0)
#comp=diff_comp(t1[-1], location_codes, tdict, beta, theta, lambda_b, dist, k)
print(np.abs(l - seven_log_likelihood(location_codes, times, beta_dep, theta_dep, beta_arr, theta_arr, k_dep, k_arr, lambda_b, dist, 0)));
print(u[0] - np.sum(sum_1),
u[1] - sum_Lambda, #np.sum(sum_Lambda),
u[2] - lambda_b * t0_dep0[-1]);
print(u[3] -seven_comp(t0_dep0[-1], location_codes, times, beta_dep, theta_dep, beta_arr, theta_arr, k_dep, k_arr, lambda_b, dist, 0))

0.0
0.0 [0. 0. 0. 0.] 0.0
0.0
